In [3]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the Titanic dataset. Replace 'titanic.csv' with your own dataset file for your project.
# Ensure the dataset file exists and is in the same directory as this jupyter notebook.
spotify_df = pd.read_csv('spotify_history.csv')

# Display the first few rows of the dataframe to ensure it is loaded correctly
display(spotify_df.columns)
display(spotify_df.head(100))

Index(['spotify_track_uri', 'ts', 'platform', 'ms_played', 'track_name',
       'artist_name', 'album_name', 'reason_start', 'reason_end', 'shuffle',
       'skipped'],
      dtype='object')

,spotify_track_uri,ts,platform,ms_played,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped
0,2J3n32GeLmMjwuAzyhcSNe,2013-07-08 02:44:34,web player,3185,"Say It, Just Say It",The Mowgli's,Waiting For The Dawn,autoplay,clickrow,False,False
1,1oHxIPqJyvAYHy0PVrDU98,2013-07-08 02:45:37,web player,61865,Drinking from the Bottle (feat. Tinie Tempah),Calvin Harris,18 Months,clickrow,clickrow,False,False
2,487OPlneJNni3NWC8SYqhW,2013-07-08 02:50:24,web player,285386,Born To Die,Lana Del Rey,Born To Die - The Paradise Edition,clickrow,unknown,False,False
3,5IyblF777jLZj1vGHG2UD3,2013-07-08 02:52:40,web player,134022,Off To The Races,Lana Del Rey,Born To Die - The Paradise Edition,trackdone,clickrow,False,False
4,0GgAAB0ZMllFhbNc3mAodO,2013-07-08 03:17:52,web player,0,Half Mast,Empire Of The Sun,Walking On A Dream,clickrow,nextbtn,False,False
...,...,...,...,...,...,...,...,...,...,...,...
95,0EUSbTvyZfo9QOcxunL9Ro,2013-07-10 07:37:29,windows,345626,In Your Atmosphere - Live at the Nokia Theatre...,John Mayer,Where the Light Is: John Mayer Live In Los Ang...,unknown,trackdone,False,False
96,2oq2dbjW3sxOY0BscMWd7Q,2013-07-10 08:28:05,windows,50952,I'll Be Alright,Passion Pit,Gossamer,unknown,unknown,False,True
97,1WETgXqmJ2NaziQhtav0pK,2013-07-10 08:30:32,windows,147536,Moth's Wings,Passion Pit,Manners,unknown,unknown,False,True
98,02omMoSwhnmDAzG2F6DFcv,2013-07-10 08:34:39,windows,245346,Let Me Hold You,Josh Krajcik,Josh Krajcik - EP,unknown,trackdone,False,False
